# Develop Aggregator: Introduce Basics

In this tutorial, we will introduce you the base class of aggregator. We will explain you how to utilize the base aggregator's communication function to exchange data between different parties. 

## Aggregator Base

The base class of the aggregator can be found in the [federatedml.framework.homo.aggregator](../../../../python/federatedml/framework/homo/aggregator/aggregator_base.py) module in FATE. In this module, you will find the AggregatorBaseClient and AggregatorBaseServer classes. The AggregatorBaseClient is responsible for sending and receiving data in the client parties (guest and host), while the AggregatorBaseServer is responsible for collecting data from all clients and broadcasting it to the server party (arbiter). With these classes, you can easily exchange data between different parties using the communication function of the base aggregator.

## About the suffix

In the FATE system, each communication between the sender and receiver is aligned using a unique string token(unique when running a component) to ensure that the correct data is sent and received. Therefore, when introducing the communication interface, you will notice that each interface has a "suffix" parameter. This suffix is the part of the token that affects the communication between the parties. To better illustrate this, let's use an example. If a client wants to send a model and uses a suffix of 'model_epoch_0', then the server wants to receive this model and calls the 'collect' function, it must match the suffix used by the client's 'send' function, which is also 'model_epoch_0', to ensure that the model is received.

## AggregatorBaseClient

The AggregatorBaseClient is a class that provides three main functions: init, send, and get.

- The init function takes one parameter, communicate_match_suffix, which allows you to specify a unique name for your aggregator instance when instantiating multiple aggregators. This can help prevent the reuse of communication tokens.

- The send function allows you to transmit arbitrary Python objects to the server by using a unique string suffix each time the function is called.

- The get function retrieves a Python object from the server, with the contents of the object being determined by the suffix used. For example, if the server sends an object A with suffix B, and you use the get function with suffix B, you will be able to receive object A.

In [ ]:
class AggregatorBaseClient(object):

    def __init__(self, communicate_match_suffix: str = None):
        ...

    def send(self, obj, suffix):
        self.communicator.send_obj(obj, suffix=suffix)

    def get(self, suffix):
        return self.communicator.get_obj(suffix=suffix)

## AggregatorBaseServer

The AggregatorBaseServer is a class that provides three main functions: init, broadcast, and collect.

- The init function takes one parameter, communicate_match_suffix, which allows you to specify a unique name for your aggregator instance when instantiating multiple aggregators. This can help prevent the reuse of communication tokens.

- The broadcast function allows you to transmit arbitrary Python objects to all clients or to specific clients. 
  - The obj parameter specifies the object to be broadcasted, and the suffix parameter determines the contents of the object being transmitted. 
  - The party_idx parameter allows you to specify which clients should receive the broadcasted object. For example, a value of -1 means that the object will be broadcasted to all clients, a single integer value specifies a single client, and a list of integer values specifies a group of clients. For example, if you have client parties A, B, and C, a value of -1 means that the object will be broadcasted to all client parties A, B, and C, a value of 1 means that the object will be sent to party B only, and a value of [0, 2] means that the object will be sent to parties A and C only.


- The collect function retrieves Python objects from clients, with the contents of the object being determined by the suffix used. For example, if all clients send their objects with suffix B, and you use the collect function with suffix B, you will be able to receive a list of their objects.

In [ ]:
class AggregatorBaseServer(object):

    def __init__(self, communicate_match_suffix=None):
        ...

    def broadcast(self, obj, suffix, party_idx=-1):
        self.communicator.broadcast_obj(obj, suffix=suffix, party_idx=party_idx)

    def collect(self, suffix, party_idx=-1):
        objs = self.communicator.get_obj(suffix=suffix, party_idx=party_idx)
        return objs

## Example

The SecureAggregatorClient and SecureAggregatorServer in the [federatedml.framework.homo.aggregator.secure_aggregator](../../../../python/federatedml/framework/homo/aggregator/secure_aggregator.py) module are implementations of the AggregatorBaseClient and AggregatorBaseServer classes, respectively. These classes are the standard aggregator implementation in the FATE homo-framework and use the functions described above for complete model/loss aggregation. If you are developing a new aggregator and need a reference implementation, we recommend that you use the secure_aggregator module as a guide. 